In [7]:
import numpy as np
import torch
import torch.nn as nn

In [18]:
class MyVIT(nn.Module):
  def __init__(self, input_shape, n_patches=7, hidden_d=8):
    super(MyVIT, self).__init__()
    self.input_shape = input_shape # (C, H, W)
    self.n_patches = n_patches
    self.patch_size = (input_shape[1] / n_patches, input_shape[2] / n_patches)
    self.input_d = int(input_shape[0] * self.patch_size[0] * self.patch_size[1])
    
    self.hidden_d = hidden_d # even if C > 1 we will keep this dimension and we pass each row vector of 16 to the linear mapper each time
    
    assert input_shape[1] % n_patches == 0, "Height mst be divisible by number of patches"
    assert input_shape[2] % n_patches == 0, "Width mst be divisible by number of patches"
    
    # 1) linear mapper
    self.linear_mapper = nn.Linear(self.input_d, self.hidden_d)
    
    # 2) classification token
    self.class_token = nn.Parameter(torch.rand(1, self.hidden_d))
    
  def forward(self, images):
    n, c, h, w = images.shape
    # reshapes the images into patches
    patches = images.reshape(n, self.n_patches ** 2, self.input_d)
    
    # running the patches into the linear mapper 
    tokens = self.linear_mapper(patches) # or embeddings
    
    # adding the classification token to the patches
    patches = torch.stack([torch.vstack((self.class_token, tokens[i])) for i in range(len(tokens))])
    
    return tokens
  
  def __call__(self, images):
    return self.forward(images)
    

In [20]:
# main program for testing purposes
if __name__ == '__main__':
  model = MyVIT(input_shape=(1, 28, 28))
  images = torch.rand(3, 1, 28, 28)
  print(model(images).shape)

torch.Size([3, 49, 8])
